# Super Resolution Convoluutional Neural Network for Image Restoration

During this project, we learn how to implement:

1. use the PSNR, MSE and SSIM image quality metrics.
2. process images using OpenCV.
3. convert between the RGB, BGR and YCrCb color spaces.
4. build deep neural networks in Keras.
5. deploy and evalute the SCRNN network.

In [8]:
import sys
import keras
import numpy
import matplotlib
import skimage
import cv2
import tensorflow as tf

In [11]:
# import some necessary packages

from keras.models import Sequential
from keras.layers import Conv2D, Input
from keras.optimizers import SGD, Adam
from skimage.measure import compare_ssim as ssim
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math
import os

# python magic function
%matplotlib inline

In [14]:
# now we are define a function for peak signal-to-noise ration (PSNR)

def psnr(target, ref):
    
    
    # assume RGB/BGR image
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    
    rmse = math.sqrt(np.mean(diff ** 2.))
    
    return 20 * math.log10(255. / rmse)

# now we defined function for mean squared error (NSE)

def mse(target, ref):
    
    # NSE is the sum of the squared difference between the two images
    
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    
    return err

# now we are create the function which combine the all the images quality matrices

def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel =True))
    
    return scores
